In [57]:
import json
from pprint import pprint
import networkx as nx

In [64]:
# read all config files and create a list of dictionaries. each config file (in json) 
# is a dictionary with keys - source uri, sink url, command, and config
dlist=[]
import glob, os
os.chdir("./")
for file in glob.glob("*.config.json"):
    with open(file) as data_file:
        data = json.load(data_file)
    dlist.append(data.copy())
    pprint(data)

{u'cmd': u'echo 01',
 u'config': None,
 u'sink.uri': [u'data.01.txt'],
 u'source.uri': [None],
 u'status': 0}
{u'cmd': u'echo 02 > data.02.txt | cat data.01.txt >> data.02.txt',
 u'config': None,
 u'sink.uri': [u'data.02.txt'],
 u'source.uri': [u'data.01.txt'],
 u'status': 0}
{u'cmd': u'echo 03 > data.03.txt | cat data.02.txt >> data.03.txt',
 u'config': None,
 u'sink.uri': [u'data.03.txt'],
 u'source.uri': [u'data.02.txt'],
 u'status': 0}


In [127]:
# create a graph where each node is a data-processing step (corresponding to a config file)
# if a node's source uri has matching entry in any other node's sink url, then ther is an edge from
# source node to sink node.
n = (len(dlist))
edge_list = []
G=nx.DiGraph()

for src in range(n):
    x = dlist[src]['source.uri']
    x1 = set([str(item) for item in x])
    print("src: " + str(src))
    if all("None" in s for s in x1):
        continue
    for snk in (set(range(n))-set([src])):
        print("\t snk: " + str(snk))
        y = dlist[snk]['sink.uri']
        y1 = set([str(item) for item in y])
        cmn = (x1 & y1)
        if cmn != set():
            print("\t\t link: "+ str(list(cmn)))
            edge_list.append([src,snk])

pprint(edge_list)

# create the Graph
G.add_nodes_from(range(n))
G.add_edges_from(edge_list)
# display the Graph
pprint(G.nodes())
pprint(G.edges())

src: 0
src: 1
	 snk: 0
		 link: ['data.01.txt']
	 snk: 2
src: 2
	 snk: 0
	 snk: 1
		 link: ['data.02.txt']
[[1, 0], [2, 1]]
[0, 1, 2]
[(1, 0), (2, 1)]


In [128]:
# check if the graph is a DAG
if nx.is_directed_acyclic_graph(G):
    print("Compute Graph is DAG")
else:
    print("something wrong with data flow")    

Compute Graph is DAG
